In [ ]:
import contextlib
import json

print('working')
import pydal.objects

try:
    import typedict
except ImportError:
    !pip install typedict
%run ~/work/__init__edwh__new.ipynb
!pip install pydal httpx[http2] trio ipywidgets chardet
from pydal import DAL, Field
import os, pathlib, httpx, trio
import typedict as typedict

if (workdir := pathlib.Path('/home/jovyan/work')).exists():
    os.chdir(workdir)
elif pathlib.Path('.jupyterlab/notebooks/DUO').exists():
    os.chdir('.jupyterlab/notebooks')
elif pathlib.Path('.').absolute().name == 'DUO':
    os.chdir('..')
elif pathlib.Path('.').absolute().name == 'notebooks':
    pass
else:
    raise ValueError(os.getcwd())
print(os.getcwd())

In [ ]:
if 'duodb' not in locals():
    duodb = pydal.DAL('sqlite://duo-data.sqlite3', folder='./DUO/db')
import slugify
from functools import partial

def fieldname(name):
    if name.isdigit():
        name = 'jaar_'+name
    return slugify.slugify(name, separator='_',replacements=[['-','_']])


[Datamodel](https://www.edustandaard.nl/app/uploads/2022/11/Canoniek-model-RIO-Generiek-20221121.pdf)

Bronnen:
https://onderwijsdata.duo.nl/datasets/rio_nfo_po_vo_vavo_mbo_ho

 * [Onderwijslocatiegebruiken](https://onderwijsdata.duo.nl/datasets/rio_nfo_po_vo_vavo_mbo_ho/resources/a86ef529-66dd-4fee-94f2-a947d4fc4617)
   `https://onderwijsdata.duo.nl/api/3/action/datastore_search?resource_id=a86ef529-66dd-4fee-94f2-a947d4fc4617&limit=5`
   > Op deze pagina vindt u de bestanden met betrekking tot de registratie instellingen en opleidingen (nfo, po, vo, vavo, mbo en ho). Deze bestanden kunt u handmatig downloaden of via de API (Application Programming Interface) te raadplegen. Voor vragen of uitgebreidere documentatie, waaronder het overkoepelend relatiemodel, kunt u contact opnemen met gegevensmagazijn@duo.nl.
 * [informatiemodel en begrippen](https://www.rio-onderwijs.nl/informatiemodel-en-begrippen)


Betere alternatieven:
https://datascience.stackexchange.com/questions/63101/collaborating-on-jupyter-notebooks

In [ ]:
# duodb.mbo_relaties_opleidingseenheden_erkenningen.truncate()
import ipywidgets as widgets
import contextlib
import json
import chardet

def autodetect(content):
    return chardet.detect(content).get("encoding")

output = widgets.Output()
display(output)


class Field(typedict.TypeDict):
    id: str
    type: str


layout = widgets.Layout(width='500px', height='40px')  #set width and height
table_progress = widgets.IntProgress(description='Tables:')
concurrent_requests = widgets.IntProgress(max=15, description='Reqs:')
table_progress.value = 0

HTTPX_LIMITS = httpx.Limits(max_connections=concurrent_requests.max, max_keepalive_connections=concurrent_requests.max)
RETRIES = 15
TIMEOUT = 90
RECORD_BATCH_SIZE = 100

with output:
    display(table_progress, concurrent_requests)

fieldmap = dict(int='integer', timestamp='datetime', numeric='float', int4='integer')


api = httpx.get('https://onderwijsdata.duo.nl/api/3/action/package_show?id=rio_nfo_po_vo_vavo_mbo_ho').json()

resource_map = {resource['name']: resource['id'] for resource in api['result']['resources']}
# resource_map = {k: v for k, v in resource_map.items() if (not k.endswith('cohorten'))} # and 'nfo' not in k
resource_map = {k: v for k, v in resource_map.items() if
                k.startswith('aangeboden') and k.endswith('opleidingen') or k in [
                    'onderwijsbesturen', 'onderwijslocatiegebruiken',
                    'onderwijslocaties',
                    'onderwijsaanbieders',
                    'relaties_onderwijsbesturen_onderwijsaanbieders',
                    'relaties_onderwijslocatiegebruiken_vestigingserkenningen',
                ]}  # and 'nfo' not in k


resource_map |= {
    'leerlingen_po_per_vestiging': '9278ae97-4014-49f4-91fc-8cc255c2595d',
    'leerlingen_vo_per_vestiging': 'd49219cc-2f36-4c2d-8007-b385ba44ec8d'
}


def define_table(db, table_name, fields: list[Field]):
    dal_fields = []
    for idx, field in enumerate(fields):
        name = 'duo_id' if (name := field['id']) == '_id' else name
        fields[idx]['id'] = name  # update the name accordingly
        dal_fields.append(pydal.Field(fieldname(name), fieldmap.get(field['type'], field['type'])))

    with contextlib.suppress(SyntaxError):
        # supress table already exists on retry
        return db.define_table(table_name, *dal_fields)

    return db[name]


async def get(url, client, output):
    for retry in range(RETRIES):
        try:
            # print('requesting', url)
            concurrent_requests.value += 1
            response = await client.get(url, timeout=TIMEOUT)
            # print('received', url)
            # print('Charset:', response.encoding)
            # print('Content-Type:', response.headers['Content-Type'])
            # for line in response.text.splitlines():
            #     if 'Australi' in line:
            #         print(line)
            #         break
            js = response.json()
            # print('received', url)
            # with output:
            #     print('+', len(js['result']['records']))
            return js
        except (httpx.HTTPError, httpx.RemoteProtocolError) as e:
            with output:
                print('retry', retry, ':', url, 'cause:', e)
        finally:
            concurrent_requests.value -= 1
    raise RuntimeError(f'Failed to get {url} after {RETRIES}')


async def load_data(db: DAL, table: pydal.objects.Table, lock: trio.Lock, pbar, fields: list[Field],
                    rows: list[list[any]]):
    field_names = [fieldname(f['id']) for f in fields]
    timestamps_field_indexes = [i for i, f in enumerate(fields) if f['type'] == 'timestamp']

    async with lock:
        table.truncate()

    bulk = []
    for row_idx, row in enumerate(rows):
        for ts_index in timestamps_field_indexes:
            row[ts_index] = value.replace('T', ' ') if (value := row[ts_index]) else None
        # if 'Australi' in str(row):
        #     print(row)
        bulk.append(dict(zip(field_names, row)))
        if row_idx % RECORD_BATCH_SIZE == 0:
            pbar.value = row_idx
            async with lock:
                table.bulk_insert(bulk)
                bulk = []
                db.commit()
    if bulk:
        async with lock:
            table.bulk_insert(bulk)
    pbar.value = row_idx + 1


async def process_table(db: DAL, table_name: str, gid: str, lock: trio.Lock, client: httpx.Client):
    table_progress.value += 1

    pbar = widgets.IntProgress(layout=layout, description=table_name, tooltip=table_name)
    display(pbar)

    try:
        js = await get(f'https://onderwijsdata.duo.nl/api/3/action/datastore_search?resource_id={gid}&limit=1', client,
                       output)
        pbar.max = estimated = js['result']['total']
        fields = js['result']['fields']
    except (json.JSONDecodeError, KeyError):
        pbar.tooltip = f'❌ {table_name}'
        pbar.description = f'❌ {table_name}'
        pbar.bar_style='danger'
        print('failed to get', table_name)
        return

    table = define_table(db, table_name, fields)
    async with lock:
        table_length_mismatch = estimated != db(table).count()
    if table_length_mismatch:
        pbar.bar_style = 'success'
        try:
            js = await get(f'https://onderwijsdata.duo.nl/datastore/dump/{gid}?format=json', client, output)
        except json.JSONDecodeError:
            return
        records = js['records']
        pbar.tooltip = f'{len(records)}:{table_name}'
        await load_data(db, table, lock, pbar, fields, records)
    else:
        pbar.bar_style = 'success'
        pbar.tooltip = f'🥫{estimated}:{table_name}'
        pbar.description = '🥫' + pbar.description
    table_progress.value += 1


async def main():
    duodb_lock = trio.Lock()
    table_progress.max = len(resource_map) * 2  # 1 for starting the table, 1 for finishing the table
    table_progress.description += f':{table_progress.max / 2}'
    async with httpx.AsyncClient(timeout=TIMEOUT, http2=True, limits=HTTPX_LIMITS,
                                 default_encoding=autodetect) as client, trio.open_nursery() as nursery:
        for idx, (name, gid) in enumerate(resource_map.items()):
            print('starting', name, gid, idx, '/', len(resource_map))
            nursery.start_soon(
                process_table, duodb, name, gid, duodb_lock, client
            )


trio.run(main)
duodb.commit()
print('done')

## Leerlingaantallen 
Dit gebeurd voor elke sector apart, en helaas zijn er hoogstens voor het MBO nu cijfers beschikbaar via RIO, voor de anderen moet er nog een koppeling gemaakt worden met elk van de scholen. 
Al deze bestanden zijn CSV bestanden (;-gescheiden) waardoor ze eenvoudig te parsen zijn. 

### PO
via https://duo.nl/open_onderwijsdata/primair-onderwijs/aantal-leerlingen/historisch-overzicht-leerlingen-schoolvestiging.jsp : 
 * https://duo.nl/open_onderwijsdata/images/06.-historisch-overzicht-aantal-leerlingen-per-schoolvestiging.csv
   - gebruikt instellingscode (brin) en nummer vestigings, wat samen de brincode per vestiging is. Dit is per jaar, zodat we eventueel grafiekjes kunnen maken. 
   - bij aantallen kan ook tekst voorkomen als "<5" 
   - bevat ook een denominatie per vestiging 

In [ ]:
import httpx, csv 

In [ ]:
def save_table(db, tablename, url):
    print('downloading',tablename, url)
    resp = httpx.get(url, follow_redirects=True)
    lines = list(resp.iter_lines())    
    print('downloaded',len(lines),'lines of csv file')
    ten_percent = len(lines)//10
    reader = csv.DictReader(lines, delimiter=';')
    for lineno, line in enumerate(reader):
        fieldname_map = {fn:fieldname(fn) for fn in reader.fieldnames}
        if tablename not in db:
            print(fieldname_map)
            print(pydal.Field)
            print(pydal.Field('brinnummer'))
            db.define_table(tablename, *[pydal.Field(fn) for fn in fieldname_map.values()])
            db[tablename].truncate()
        record = {fieldname_map[key]:value for key,value in line.items()}
        db[tablename].insert(**record)
        if lineno % ten_percent == 0: 
            print(lineno, round(100*lineno/len(lines)),'%')
    print('100%')
    db.commit()
save_table(duodb, 'po_aantallen_bron', 'https://duo.nl/open_onderwijsdata/images/06.-historisch-overzicht-aantal-leerlingen-per-schoolvestiging.csv')

In [ ]:
duodb.executesql('''drop view if exists po_aantallen;''')
duodb.executesql('''
create view po_aantallen as
select brinnummer||po_aantallen_bron.vestigingsnummer as vestigingsnummer,
       round(sum(case when leerlingen_2022 = '<5' then 2.5 else cast(leerlingen_2022 as integer) end)) as aantal_2022,
       round(sum(case when leerlingen_2021 = '<5' then 2.5 else cast(leerlingen_2021 as integer) end)) as aantal_2021,
       round(sum(case when leerlingen_2020 = '<5' then 2.5 else cast(leerlingen_2020 as integer) end)) as aantal_2020,
       round(sum(case when leerlingen_2019 = '<5' then 2.5 else cast(leerlingen_2019 as integer) end)) as aantal_2019
       from po_aantallen_bron
    group by brinnummer, vestigingsnummer
''')
duodb.executesql('''drop table if exists po_aantallen_materialized;''')
duodb.executesql('''create table po_aantallen_materialized as select * from po_aantallen;''')


### VO 
via https://duo.nl/open_onderwijsdata/voortgezet-onderwijs/aantal-leerlingen/aantal-leerlingen.jsp: 
 * https://duo.nl/open_onderwijsdata/images/03.-leerlingen-vo-per-vestiging-en-bestuur-(vavo-apart)-2022.csv
   - gebruikt "vestigingsnummer", ofwel de brin-locatie
   

In [ ]:
save_table(duodb, 'vo_aantallen_bron', 'https://duo.nl/open_onderwijsdata/images/03.-leerlingen-vo-per-vestiging-en-bestuur-(vavo-apart)-2022.csv')

In [ ]:
duodb.executesql('''drop view if exists vo_aantallen;''')
duodb.executesql('''
create view vo_aantallen as
select vestigingsnummer, totaal_aantal_leerlingen as aantal from vo_aantallen_bron
''')
duodb.executesql('''drop table if exists vo_aantallen_materialized;''')
duodb.executesql('''create table vo_aantallen_materialized as select * from vo_aantallen;''')


### MBO
Via https://duo.nl/open_onderwijsdata/middelbaar-beroepsonderwijs/aantal-studenten/studenten-mbo-rio.jsp: 
 * https://duo.nl/open_onderwijsdata/images/onderwijslocaties-mbo-met-inschrijvingen-en-geografische-gegevens-per-1-oktober-2022.csv 
   - gebruikt o.a. de kolom `onderwijslocatie`, wat komt uit de RIO, dus daar is makkelijk op te koppelen 

In [ ]:
save_table(duodb, 'mbo_aantallen_bron', 'https://duo.nl/open_onderwijsdata/images/onderwijslocaties-mbo-met-inschrijvingen-en-geografische-gegevens-per-1-oktober-2022.csv')

In [ ]:
duodb.executesql('''drop view if exists mbo_aantallen;''')
duodb.executesql('''
create view mbo_aantallen as
select onderwijslocatie, aantal_hoofdinschrijvingen as aantal from mbo_aantallen_bron
''')
duodb.executesql('''drop table if exists mbo_aantallen_materialized;''')
duodb.executesql('''create table mbo_aantallen_materialized as select * from mbo_aantallen;''')


### HO
Via https://duo.nl/open_onderwijsdata/hoger-onderwijs/aantal-studenten/studenten-hbo.jsp: 
 * https://duo.nl/open_onderwijsdata/images/01a.-ingeschrevenen-hbo-2022.csv 
 

In [ ]:
save_table(duodb, 'ho_aantallen_bron', 'https://duo.nl/open_onderwijsdata/images/01a.-ingeschrevenen-hbo-2022.csv')

In [ ]:
duodb.executesql('''drop view if exists ho_aantallen;''')
duodb.executesql('''
create view ho_aantallen as
select brin_nummer_actueel as brin,
       round(sum(case when jaar_2022 = '<5' then 2.5 else cast(jaar_2022 as integer) end)) as aantal_2022,
       round(sum(case when jaar_2021 = '<5' then 2.5 else cast(jaar_2021 as integer) end)) as aantal_2021,
       round(sum(case when jaar_2020 = '<5' then 2.5 else cast(jaar_2020 as integer) end)) as aantal_2020,
       round(sum(case when jaar_2019 = '<5' then 2.5 else cast(jaar_2019 as integer) end)) as aantal_2019
  from ho_aantallen_bron
 group by brin_nummer_actueel
''')
duodb.executesql('''drop table if exists ho_aantallen_materialized;''')
duodb.executesql('''create table ho_aantallen_materialized as select * from ho_aantallen;''')


### WO 
via https://duo.nl/open_onderwijsdata/hoger-onderwijs/aantal-studenten/studenten-wo.jsp: 
  * https://duo.nl/open_onderwijsdata/images/04.-inschrijvingen-wo-2022.csv 
    - ook hier alleen  BRIN nummers, eventueel zou een koppeling op studienummer wellicht mogelijk zijn, maar dat is voor ons niet zo boeiend, want WO doen we nu toch amper wat mee. 

In [ ]:
save_table(duodb, 'wo_aantallen_bron', 'https://duo.nl/open_onderwijsdata/images/04.-inschrijvingen-wo-2022.csv')

In [ ]:
duodb.executesql('''drop view if exists wo_aantallen;''')
duodb.executesql('''
create view wo_aantallen as
select brin_nummer_actueel as brin,
       round(sum(case when jaar_2022 = '<5' then 2.5 else cast(jaar_2022 as integer) end)) as aantal_2022,
       round(sum(case when jaar_2021 = '<5' then 2.5 else cast(jaar_2021 as integer) end)) as aantal_2021,
       round(sum(case when jaar_2020 = '<5' then 2.5 else cast(jaar_2020 as integer) end)) as aantal_2020,
       round(sum(case when jaar_2019 = '<5' then 2.5 else cast(jaar_2019 as integer) end)) as aantal_2019
  from wo_aantallen_bron
 group by brin_nummer_actueel

''')
duodb.executesql('''drop table if exists wo_aantallen_materialized;''')
duodb.executesql('''create table wo_aantallen_materialized as select * from wo_aantallen;''')


In [ ]:
for statement in [_.strip() for _ in '''
drop view if exists aangeboden_opleiding;
create view aangeboden_opleiding as
with source_tables as (select onderwijslocatiecode, onderwijsaanbiederid, 'aangeboden_bo_opleidingen' as ao_table
                       from aangeboden_bo_opleidingen
                       union all
                       select onderwijslocatiecode, onderwijsaanbiederid, 'aangeboden_ho_opleidingen' as ao_table
                       from aangeboden_ho_opleidingen
                       union all
                       select onderwijslocatiecode, onderwijsaanbiederid, 'aangeboden_mbo_opleidingen' as ao_table
                       from aangeboden_mbo_opleidingen
                       union  all
                       select onderwijslocatiecode, onderwijsaanbiederid, 'aangeboden_so_opleidingen' as ao_table
                       from aangeboden_so_opleidingen
                       union all
                       select onderwijslocatiecode, onderwijsaanbiederid, 'aangeboden_vavo_opleidingen' as ao_table
                       from aangeboden_vavo_opleidingen
                       union all
                       select onderwijslocatiecode, onderwijsaanbiederid, 'aangeboden_vo_opleidingen' as ao_table
                       from aangeboden_vo_opleidingen
                       union all
                       select onderwijslocatiecode, onderwijsaanbiederid, 'aangeboden_vso_opleidingen' as ao_table
                       from aangeboden_vso_opleidingen
                       union all
                       select onderwijslocatiecode, onderwijsaanbiederid, 'aangeboden_nfo_opleidingen' as ao_table
                       from aangeboden_nfo_opleidingen)
select onderwijslocatiecode, onderwijsaanbiederid, group_concat(ao_table, ',') as sources
from source_tables
group by onderwijslocatiecode, onderwijsaanbiederid;
drop index if exists olg_idx;
create unique index olg_idx on onderwijslocatiegebruiken (onderwijslocatiecode, onderwijsbestuurid, uitbedrijfdatum);
drop index if exists oa_idx;
create index oa_idx on onderwijsaanbieders (onderwijsaanbiederid, naam, einddatum_periode);
drop index if exists ol_idx;
create unique index ol_idx on onderwijslocaties (onderwijslocatiecode, gps_latitude, gps_longitude, plaatsnaam, straatnaam, huisnummer, huisnummertoevoeging, postcode);
drop index if exists ob_idx;
create index ob_idx on onderwijsbesturen (onderwijsbestuurid, naam);
drop table if exists aangeboden_opleidingen_materialized;
create table aangeboden_opleidingen_materialized as
select onderwijsaanbiederid, onderwijslocatiecode, sources
from aangeboden_opleiding
group by onderwijslocatiecode;
drop index if exists ao_idx;
create unique index ao_idx on aangeboden_opleidingen_materialized (onderwijslocatiecode, onderwijsaanbiederid, sources);

drop index if exists roa_idx;
create index roa_idx on relaties_onderwijsbesturen_onderwijsaanbieders (onderwijsaanbiederid, onderwijsbestuurid);
drop index if exists rov_idx;
create index rov_idx on relaties_onderwijslocatiegebruiken_vestigingserkenningen (onderwijslocatiecode);
drop view if exists edwh_onderwijslocaties;
create view edwh_onderwijslocaties as 
select case
           when oa.naam is null and olg.uitbedrijfdatum is not null then 'Niet langer gebruikt'
           when oa.naam is null and olg.uitbedrijfdatum is null then 'Geen opleiding geregistreerd'
           else oa.naam end                                                                                   as naam
     , group_concat(ob.naam, '; ')                                                                            as besturen
     , ol.straatnaam
     , ol.huisnummer
     , ol.huisnummertoevoeging
     , ol.postcode
     , ol.plaatsnaam
     , ol.gps_latitude
     , ol.gps_longitude
     , ol.onderwijslocatiecode
     , replace(replace(ao.sources, 'aangeboden_', ''), '_opleidingen', '')                                    as sources
     , group_concat(distinct rov.vestigingscode)                                                              as vestiging_brins
     , case
           when coalesce(olg.uitbedrijfdatum, oa.einddatum_periode, 'Y') = 'Y' then 'Y'
           else 'N' end                                                                                       as in_bedrijf
     , coalesce(sum(mbo_aantallen.aantal), 0)
    + coalesce(sum(po_aantallen.aantal_2022), 0)
    + coalesce(sum(vo_aantallen.aantal), 0)
    + coalesce(sum(ho_aantallen.aantal_2022), 0)
    + coalesce(sum(wo_aantallen.aantal_2022), 0)
                                                                                                              as aantal_leerlingen_per_vestiging
     , coalesce(sum(mbo_aantallen.aantal), 0)
    + coalesce(sum(po_aantallen.aantal_2022), 0)
    + coalesce(sum(vo_aantallen.aantal), 0)
    + coalesce(sum(ho_aantallen.aantal_2022), 0)
    + coalesce(sum(wo_aantallen.aantal_2022), 0)                                                              as aantal_leerlingen_per_organisatie
from onderwijslocatiegebruiken olg
         left outer join onderwijslocaties ol
                         on olg.onderwijslocatiecode = ol.onderwijslocatiecode
         left outer join onderwijsbesturen ob
                         on olg.onderwijsbestuurid = ob.onderwijsbestuurid and ob.einddatum_periode is null
         left outer join aangeboden_opleidingen_materialized ao on ol.onderwijslocatiecode = ao.onderwijslocatiecode
         left outer join onderwijsaanbieders oa
                         on oa.onderwijsaanbiederid = ao.onderwijsaanbiederid -- and oa.einddatum_periode is null
         left outer join relaties_onderwijsbesturen_onderwijsaanbieders roa
                         on roa.onderwijsaanbiederid = oa.onderwijsaanbiederid
                             and roa.onderwijsbestuurid = ob.onderwijsbestuurid
         left outer join relaties_onderwijslocatiegebruiken_vestigingserkenningen rov
                         on rov.onderwijslocatiecode = ol.onderwijslocatiecode
         left outer join mbo_aantallen_materialized as mbo_aantallen on mbo_aantallen.onderwijslocatie = ol.onderwijslocatiecode
         left outer join po_aantallen_materialized as po_aantallen on po_aantallen.vestigingsnummer = rov.vestigingscode
         left outer join vo_aantallen_materialized as vo_aantallen on vo_aantallen.vestigingsnummer = rov.vestigingscode
         left outer join ho_aantallen_materialized as ho_aantallen on rov.vestigingscode like ho_aantallen.brin || '%'
         left outer join wo_aantallen_materialized as wo_aantallen on rov.vestigingscode like wo_aantallen.brin || '%'
-- where ol.plaatsnaam = 'Assen' -- ###########################
group by oa.naam
       , oa.onderwijsaanbiederid
       , ol.straatnaam
       , ol.huisnummer
       , ol.huisnummertoevoeging
       , ol.postcode
       , ol.plaatsnaam
       , ol.gps_latitude
       , ol.gps_longitude
       , ol.onderwijslocatiecode
       , rov.onderwijslocatiecode
'''.split(';\n')]:
    print(statement)
    print('result:',duodb.executesql(statement))
    print()
    

In [ ]:
duodb.commit()

In [ ]:
%%sql sqlite:///DUO/db/duo-data.sqlite3
select * from edwh_onderwijslocaties limit 10;

In [ ]:
!ls -alh DUO/db/*.sqlite3


In [ ]:
#originele_import = %sql sqlite:///DUO/all_with_gid.db select * from scholen